<a href="https://colab.research.google.com/github/Korsholm22/M4_Group_Assignments/blob/main/Group_Assignment_3/NHN_SBERT_Netflix_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [15]:
!pip install sentence-transformers -q
!pip install gradio -q

In [16]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
from sentence_transformers.util import cos_sim

model = SentenceTransformer('bert-base-nli-mean-tokens')
#model = SentenceTransformer('all-mpnet-base-v2')

model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [17]:
# Importing the dataset
df_netflix = pd.read_csv('https://raw.githubusercontent.com/Korsholm22/M4_Group_Assignments/main/Group_Assignment_3/Data/netflix_titles.csv')

In [18]:
# Examining the dataset
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [19]:
search = ["Action movie taking place in space",    
             "Romantic drame where the man proposes to the woman",    
             "Sad movie where the dog dies",    
             "Funny movie taking place in the jungle",    
             "True crime series where the detective gets hurt while hunting the bad guys"]

In [20]:
embeddings = model.encode(search)

embeddings.shape

(5, 768)

In [21]:
sim = np.zeros((len(search), len(search)))

for i in range(len(search)):
    sim[i:,i] = cos_sim(embeddings[i], embeddings[i:])

sim

array([[1.00000024, 0.        , 0.        , 0.        , 0.        ],
       [0.10741168, 1.00000012, 0.        , 0.        , 0.        ],
       [0.11696689, 0.21837732, 1.        , 0.        , 0.        ],
       [0.33412543, 0.17495906, 0.42840776, 1.        , 0.        ],
       [0.07026712, 0.16477531, 0.53109002, 0.34692168, 1.        ]])

In [22]:
show_descriptions = df_netflix['description'].values.tolist()

In [23]:
show_descriptions

['As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.',
 'After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.',
 'To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.',
 'Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.',
 'In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.',
 'The arrival of a charismatic young priest brings glorious miracles, ominous mysteries and renewed religious fervor to a dying town desperate to believe.',
 "Equestria's divided. But a bright-eyed hero believes Earth Ponies, Pegas

In [ ]:
corpus_embeddings = model.encode(show_descriptions, convert_to_tensor=True)

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(show_descriptions))
for query in search:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(show_descriptions[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """





Query: Action movie taking place in space

Top 5 most similar sentences in corpus:
After landing on a planet reminiscent of 1950s suburbia, a human astronaut tries to avoid capture, recover his spaceship and make it home safely. (Score: 0.7683)
A mysterious game sends two brothers flying into space, where they must reach the planet Zathura — or be trapped in galactic limbo forever. (Score: 0.7561)
With access to the scientists and engineers responsible for the Curiosity rover's on-the-ground experiments, NOVA captures its landing on Mars. (Score: 0.7386)
Two astronauts attempt to brave a life in Earth's orbit on a record-setting mission to see if humans have the endurance to survive a flight to Mars. (Score: 0.7324)
After falling through a wormhole, a space-dwelling teen mecha pilot awakes to find himself amid the inhabitants of a strange planet called ... Earth. (Score: 0.7318)




Query: Romantic drame where the man proposes to the woman

Top 5 most similar sentences in corpus:
A

In [ ]:
def query_corpus(query):
  results = []
  query_embedding = model.encode(query, convert_to_tensor=True)
  cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    results.append((show_descriptions[idx], score))
    output = results[0][0]
    score = float(results[0][1].numpy())
  return output, score

In [ ]:
query_corpus('Action movie with batman')

("When Superman and the other Justice League superheroes start disappearing, it's up to Batman to unravel the caper and save the day.",
 0.5879056453704834)

In [ ]:
import gradio as gr

gr.Interface(fn=query_corpus, inputs=["text"], outputs=["text", "label"]).launch()